In [1]:
import numpy as np
from math import log

def split_data(data, n_tr=5, n_val=1):
    m = n_tr+n_val+1
    N = data.shape[0]
    indices = np.random.permutation(N)
    training_idx, val_idx, test_idx = indices[:int(n_tr * N/m)], indices[int(n_tr * N/m):int((n_tr+n_val) * N/m)], indices[int((n_tr+n_val) * N/m):]   
    training, validation, test = data[training_idx,:], data[val_idx,:], data[test_idx,:]
    return training, validation, test



In [104]:
class Partition(object):
    """
    Object for data partition b y feature of index feature_idx
    """
    
    def __init__(self, X, feature_idx, possible_classes):
        self.feature_idx = feature_idx # for debug
        self.possible_classes = possible_classes
        self.possible_feature_values = np.unique(X[:,feature_idx])
        self.decision_map = {value: [] for value in self.possible_feature_values}
        self.N = len(X)
        for x in X:
            self.decision_map[x[feature_idx]].append(x)
    
    def score(self):
        result = 0
        for key in self.decision_map.keys():
            key_result = 0
            for cls in self.possible_classes:
                p_class = sum(map(lambda x: x[-1] == cls, self.decision_map[key]))/len(self.decision_map[key])
                if(p_class):
                    key_result += -p_class * log(p_class)
            result += key_result * (len(self.decision_map[key])/self.N)
        return result
    
    def print_(self):
        print('Feature index: %s' % self.feature_idx)
        for k, v in self.decision_map.items():
            print('Elements for value %s:' % k)
            for row in v:
                print('    ' + str(row))
        
        
        
        

In [136]:
def find_greatest_class(X):
    count_dict = {}
    for x in X:
        if x[-1] in count_dict.keys():
            count_dict[x[-1]] +=1
        else:
            count_dict[x[-1]] = 1
    return sorted(count_dict.items(), key=lambda x: x[-1])[-1][0]

class DecisionTree(object):
    def __init__(self, X, possible_classes, log=False):
        self.nodes = {}
        self.class_ = None
        self.classes = np.unique(X[:,-1])
        self.possible_classes = possible_classes
        self.classes_map = {}
        self.X = X
        self.log = log
        if self.log:
            print(X)
            print(X[:,-1])
        if np.all(np.all(X[:,:-1] == X[:,:-1][0,:], axis = 1)):
            self.class_ = find_greatest_class(X)
            return
        if len(self.classes) == 1:
            self.class_ = self.classes[0]
            return
        
        min_value = self.find_best_partition(X)
        self._build_node()
        
    def find_best_partition(self, X):
        best_partition = None
        min_value = 100
        for feature_idx in range(len(X[0])-1):
            part = Partition(X, feature_idx, self.possible_classes)
            score = part.score()
            if score < min_value:
                min_value = score
                best_partition = part
                if self.log:
                    print(best_partition)
        self.partition = best_partition
        self.feature_idx = best_partition.feature_idx
        return min_value

    def _build_node(self):
        for k, v in self.partition.decision_map.items():
            self.nodes[k] = DecisionTree(np.array(v), self.possible_classes)
    
    
    def classify_single(self, x):        
        if not self.class_:
            if not hasattr(self, 'feature_idx'):
                return self.get_class()
            if str(x[self.feature_idx]) in self.nodes:
                return self.nodes[str(x[self.feature_idx])].classify_single(x)
            else:
                return find_greatest_class(self.X)
        else:
            return self.class_
    
    def classify(self, X):
        result = [self.classify_single(x) for x in X]
        return sum(result[i] == X[i][-1] for i in range(len(X)))/len(X)
    
    def is_leaf(self):
        return bool(self.class_)
    
    def print_(self):
        if self.is_leaf():
            print('Leaf. class: %s' % self.class_)
        else:
            self.partition.print_()
            print('Children-------------------------------------\n')
            if self.nodes:
                for v in self.nodes.values():
                    v.print_()
            print('\n\n\n')
            
    def find_nodes_with_leafs(self):
        res = []
        for k, v in self.nodes.items():
            if v.has_only_leafs():
                res.append(v)
            elif not v.is_leaf():
                res.extend(v.find_nodes_with_leafs())
        return res
    
    def prune(self, validation):
        nodes_with_leafs = self.find_nodes_with_leafs()
        for n in nodes_with_leafs:
            acc = self.classify(validation)
            if self.log:
                print(n.X, find_greatest_class(n.X))
                print(acc, self.classify(validation))
            n.class_ = find_greatest_class(n.X)
            if self.classify(validation) < acc:
                n.class_ = None
                
    def has_only_leafs(self):
        for k,v in self.nodes.items():
            if not v.is_leaf():
                return False
        return True

    def get_class(self):
        if hasattr(self, 'class_'):
            return self.class_
        else:
            print(X)
            return find_greatest_class(X)
            

In [196]:
def decision_tree_test(data):
    train, validate, test = split_data(data, 5, 2)
    print('Building decision Tree...')
    d = DecisionTree(train, np.unique(data[:,-1]))

    print('Score on validate after training: ', d.classify(validate))
    print('Score on test after training: ', d.classify(test))
    d.prune(validate)
    print('Score on validate after pruning: ', d.classify(validate))
    print('Score on test after training: ', d.classify(test))

    
def load_data(path, separator):
    with open(path, 'r') as f:
        content = f.readlines()
        return [list(filter(lambda x: x, x.strip().split(separator))) for x in content]

In [197]:
def load_lenses(): 
    return np.roll(np.array(load_data('../lenses/lenses.data', ' ')), -1)[:,: -1]

def load_hayes():
    return np.roll(np.array(load_data('../hayes/hayes-roth.data', ',')), -1)[:,:-1]

def load_mushroom():
    return np.roll(np.array(load_data('../mushroom/agaricus-lepiota.data', ',')), -1)

In [205]:
print('Test DecisionTree on Lenses Database:\n')
decision_tree(load_lenses())
print('\n\n')
print('Test DecisionTree on Hayes Database:\n')
decision_tree(load_hayes())
print('\n\n')
print('Test DecisionTree on Mushroom Database:\n')
decision_tree_test(load_mushroom()[:500])

Test DecisionTree on Lenses Database:

Building decision Tree...
Score on validate after training:  0.6666666666666666
Score on test after training:  1.0
Score on validate after pruning:  0.6666666666666666
Score on test after training:  1.0



Test DecisionTree on Hayes Database:

Building decision Tree...
Score on validate after training:  0.7272727272727273
Score on test after training:  0.7647058823529411
Score on validate after pruning:  0.7272727272727273
Score on test after training:  0.7647058823529411



Test DecisionTree on Mushroom Database:

Building decision Tree...
Score on validate after training:  0.816
Score on test after training:  0.6825396825396826
Score on validate after pruning:  0.864
Score on test after training:  0.7301587301587301
